In [1]:
import ast
from collections import defaultdict
from datetime import datetime
import numpy as np
import pandas as pd
from pocketknife.database import connect_database, read_from_database
from config import EXTERNAL_DATA_DIR

2025-03-05 10:22:24.699 | INFO     | config:<module>:11 - PROJ_ROOT path is: /Users/manuelbolz/Documents/git/for_work/company_success_prediction


# LOAD DATA

In [2]:
# db queries
query_shab = """
    SELECT
        ehraid,
        shab_id,
        shab_date,
        registry_office_canton,
        message AS message_raw
    FROM zefix.shab
"""

query_shab_mutation = """
    SELECT * 
    FROM zefix.shab_mutation
"""

In [3]:
with connect_database() as con:
    raw_shab = read_from_database(con, query_shab)
    raw_shab_mutation = read_from_database(con, query_shab_mutation)

In [4]:
parsed_shab_messages = pd.read_csv(EXTERNAL_DATA_DIR / 'rule_based_parsing_fr.csv')

In [5]:
parsed_shab_messages['parsed_variables'] = parsed_shab_messages.parsed_variables.fillna('{}')
parsed_shab_messages['parsed_variables'] = parsed_shab_messages.parsed_variables.apply(ast.literal_eval)
parsed_shab_messages['parsed_variables'] = parsed_shab_messages.parsed_variables.fillna({})

In [ ]:
parsed_shab_messages['text_slice'] = parsed_shab_messages['text_slice'].fillna('[]')
parsed_shab_messages['text_slices'] = parsed_shab_messages['text_slice'].apply(ast.literal_eval)
parsed_shab_messages = parsed_shab_messages.drop(columns=['text_slice'])
parsed_shab_messages.head()

In [6]:
print(parsed_shab_messages['shab_id'].nunique())
print(raw_shab['shab_id'].nunique())

636019
2464457


In [ ]:
1625920
2457792

In [7]:
# Filter out the companies where the shab entries have been parsed
raw_shab = raw_shab[raw_shab.shab_id.isin(parsed_shab_messages['shab_id'].unique())].copy()
raw_shab_mutation = raw_shab_mutation[raw_shab_mutation.shab_id.isin(parsed_shab_messages['shab_id'].unique())].copy()
raw_shab_mutation_grouped = (
    raw_shab_mutation
    .groupby('shab_id')
    .agg(codes=pd.NamedAgg(column='description', aggfunc=lambda x: [v for v in x]))
    .reset_index()
)

In [ ]:
parsed_shab_messages

In [8]:
# Merge all the dataframes
shab_merged = (
    raw_shab
    .merge(raw_shab_mutation_grouped, on='shab_id', how='left')
    .merge(parsed_shab_messages, on='shab_id', how='left')
)

In [9]:
# Sort values in the correct temporal order
shab_merged = shab_merged.sort_values(['ehraid', 'shab_date', 'shab_id'], ascending=[True, True, True]).reset_index(drop=True)

In [ ]:
shab_merged_temp = shab_merged[shab_merged.ehraid.isin([905876, 905843, 905844])]
shab_merged_temp = shab_merged_temp.sort_values(['ehraid', 'shab_date', 'shab_id'], ascending=[True, True, True]).reset_index()

In [ ]:
def create_raw_json_history(df: pd.DataFrame) -> dict:
    json_structure = defaultdict(lambda: {'history': []})
    for _, row in df.iterrows():
        # Extract the fields to identify a company and its entries
        ehraid = row['ehraid']
        shab_date = row['shab_date'] if isinstance(row['shab_date'], str) else row['shab_date'].strftime('%Y-%d-%m')
        shab_id = row['shab_id']
        main_group = row['main_group']
        keyword = row['keyword']

        # Extract main information
        message_info = {
            'registry_office_canton': row['registry_office_canton'],
            'codes': row['codes'],
            'message_raw': row['message_raw'],
            'extracted_content': {
                main_group: {
                    keyword: {
                        'text_slices': row['text_slices'],
                        'variables': row['parsed_variables']
                    }
                }
            }
        }

        # Search if shab_date already exists in the history
        date_entry = next((entry for entry in json_structure[ehraid]['history'] if shab_date in entry), None)

        if date_entry is None:
            # If the date does not exist, create a new entry
            date_entry = {shab_date: {shab_id: message_info}}
            json_structure[ehraid]['history'].append(date_entry)
        else:
            # If the date exists, check if the shab_id already exists
            id_entry = date_entry[shab_date].get(shab_id, None)
            if id_entry is None:
                # If the shab_id does not exists, we can simply add it to the shab_date
                date_entry[shab_date][shab_id] = message_info
            else:
                # If the shab_id exists, we need to check if the main_group already exists
                main_group_entry = date_entry[shab_date][shab_id]['extracted_content'].get(main_group, None)
                if main_group_entry is None:
                    # If it does not exist, we add it to the extracted content
                    date_entry[shab_date][shab_id]['extracted_content'][main_group] = message_info['extracted_content'][main_group]
                else:
                    # If it does, we add the keyword to the main_group, since a keyword can only appear once within the main_group
                    date_entry[shab_date][shab_id]['extracted_content'][main_group][keyword] = message_info['extracted_content'][main_group][keyword]      

    return dict(json_structure)

In [ ]:
history_json = create_raw_json_history(shab_merged)

# CREATE REGISTERED PEOPLE AND FIRMS TABLE

In [161]:
df_people = shab_merged[shab_merged.main_group == 'natural persons and legal entities'].copy()

In [162]:
# Make sure only the expected fields are there
def validate_person_and_firms(x: dict):    
    schema = {'firms': [], 'people': []}
    for firm in x.get('firms', []):
        if isinstance(firm, dict):
            schema['firms'].append({
                'firm_name': firm.get('firm_name'),
                'firm_uid': firm.get('id'),
                'firm_seat': firm.get('location'),
                'firm_type': firm.get('type'),
                'firm_shares': firm.get('shares')
            })
    for person in x.get('people', []):
        if isinstance(person, dict):
            schema['people'].append({
                'first_name': person.get('first_name'),
                'last_name': person.get('last_name'),
                'hometown': person.get('hometown'),
                'place_of_residence': person.get('place_of_residence'),
                'nationality': person.get('nationality'),
                'job_title': person.get('job_title'),
                'authorization': person.get('authorization'),
                'shares': person.get('shares')
            })
    return schema

df_people['validated_variables'] = df_people['parsed_variables'].apply(validate_person_and_firms)  
df_people['firms'] = df_people['validated_variables'].apply(lambda x: x.get('firms', []))
df_people['people'] = df_people['validated_variables'].apply(lambda x: x.get('people', []))

# Split individual firm dictionaries into individual rows
df_firms_exploded = df_people.explode(column=['firms']).dropna()
df_people_exploded = df_people.explode(column=['people']).dropna()

# Create individual columns from the dictionary
df_firms_norm = pd.json_normalize(
    df_firms_exploded['firms'],
    errors='raise'
)
df_firms_concat = pd.concat([df_firms_exploded[['ehraid', 'message_raw', 'shab_date', 'shab_id', 'codes', 'keyword']].reset_index(drop=True), df_firms_norm], axis=1)

df_people_norm = pd.json_normalize(
    df_people_exploded['people'],
    errors='raise'
)
df_people_concat = pd.concat([df_people_exploded[['ehraid', 'message_raw', 'shab_date', 'shab_id', 'codes', 'keyword']].reset_index(drop=True), df_people_norm], axis=1)

# Add Gender and Nationality to the data

In [80]:
import re
import spacy
from unidecode import unidecode
from zefix_processing.country_mapping import german2alpha2, french2alpha2, italian2alpha2
from zefix_processing.gender_mapping import german2gender, french2gender, italian2gender

nlp_models = {
    "de": spacy.load("de_core_news_sm"),
    "fr": spacy.load("fr_core_news_sm"),
    "it": spacy.load("it_core_news_sm")
}

In [163]:
def normalize_words(string: str) -> str:
    replacements = {
        'ä': 'ae',
        'ö': 'oe',
        'ü': 'ue'
    }
    for char, replacement in replacements.items():
        string = string.replace(char, replacement)
    
    return unidecode(string.lower())

In [164]:
nationality_mapping = {
    'de': {normalize_words(k): v for k, v in german2alpha2.items()},
    'fr': {normalize_words(k): v for k, v in french2alpha2.items()},
    'it': {normalize_words(k): v for k, v in italian2alpha2.items()}
}

gender_mapping = {
    'de': {normalize_words(k): v for k, v in german2gender.items()},
    'fr': {normalize_words(k): v for k, v in french2gender.items()},
    'it': {normalize_words(k): v for k, v in italian2gender.items()},
}

### Clean the hometown, place of residence, and nationality column

In [165]:
countries_containing_and = [
    'bosnie et herzégovine',
    'svalbard et île jan mayen',
    'îles turques et caïques',
    'géorgie du sud et îles sandwich du sud',
    'bonaire, saint eustatius et saba',
    'terres australes et antarctiques françaises',
    'de são tomé e príncipe',
    'serbo e montenegrino',
    'serba e montenegrina',
    'di saint christopher e nevis',
    'de são tomé e príncipe'
]

# 1. Split multiple nationalities into individual columns
def split_locations(df: pd.DataFrame, orig_col: str = 'nationality'):
    loc_split = df[orig_col].str.split(r'\sund\s|\set\s|\se\s', regex=True, expand=True)
    loc_split.columns = [f'{orig_col}_{i+1}' for i in range(loc_split.shape[1])]
    loc_split.fillna('', inplace=True)
    df = pd.concat([df, loc_split], axis=1)
    return df.drop(columns=[orig_col])

# First, split the hometown and place of residence column
df_people_concat = split_locations(df_people_concat, 'hometown')
df_people_concat = split_locations(df_people_concat, 'place_of_residence')

In [166]:
# 2. Move nationalities that are in the wrong column
def contains_target_word(text):
    pattern = r'\bstaatsangehoerige\b|\bcittadina\b|\bressortissante\b|\bcitoyenne\b|\bstaatsangehoeriger\b|\bcittadino\b|\bressortissant\b|\bcitoyen\b'
    return bool(re.search(pattern, text))

def move_nationalities(language: str, entries: list[str], nat: str, country_names: set[str]):
    """
    Checks if any of the hometown columns contains a country name
    """
    mapping = {
        'de': 'und',
        'fr': 'et',
        'it': 'e'
    }
    for i, entry in enumerate(entries):
        if entry:
            entry_norm = normalize_words(entry)
            if entry_norm in country_names or contains_target_word(entry_norm):
                nat = f"{nat} {mapping.get(language, ' und ')} {entry}" if nat else entry
                entries[i] = ''
            else:
                # If the name is not a country name and the nationality does not include Swiss yet,
                # we want to add 'CH' to the nationalities, since the hometown is with high probability a Swiss municipality
                if 'CH' not in nat:
                    nat = f"{nat} {mapping.get(language, ' und ')} CH" if nat else 'CH'
    return entries + [nat]

countries_norm = french2alpha2.keys()
hometown_cols = [col for col in df_people_concat.columns if 'hometown' in col]
result_cols = hometown_cols + ['nationality']
df_people_concat[result_cols] = df_people_concat.apply(lambda x: pd.Series(move_nationalities('fr', [x[col] for col in hometown_cols], x['nationality'], countries_norm)), axis=1)

In [167]:
# Finally, split the nationality column
df_people_concat = split_locations(df_people_concat, 'nationality')

### Clean the authorization and shares column

In [169]:
def switch_auth_and_shares(language: str, auth: str, shares: str):
    """
    Checks if any of the hometown columns contains a country name
    """
    keyword_mapping = {
        'de': 'unterschrift',
        'fr': 'signature',
        'it': 'firma'
    }
    and_mapping = {
        'de': 'und',
        'fr': 'et',
        'it': 'e'
    }

    # Base Case: no value in both
    if not (auth or shares):
        return [auth, shares]  # no switch
    
    match_auth = re.search(r'\bchf\b', auth.lower()) if auth else None
    match_shares = re.search(keyword_mapping[language], shares.lower()) if shares else None

    # Case 0: no match in both
    if not (match_auth or match_shares):
        return [auth, shares]  # no switch

    # Case 1: match in auth and no match in shares
    elif match_auth and not match_shares:
        if shares:
            return ['', f"{auth} {and_mapping[language]} {shares}"]  # Add auth infront of shares
        else:
            return ['', auth]  # switch columns: auth, share
    
    # Case 2: match in shares and no value in auth
    elif match_shares and not match_auth:
        if shares:
            return [f"{auth} {and_mapping[language]} {shares}", '']  # Add shares after auth
        else:
            return [shares, '']  # switch columns: auth, share
    
    # Case 3: match in auth and shares
    elif match_auth and not match_shares:
        return [shares, auth]  # switch both
    
    else:
        return [auth, shares]

In [172]:
df_people_concat[['authorization', 'shares']] = df_people_concat.apply(lambda x: pd.Series(switch_auth_and_shares('fr', x['authorization'], x['shares'])), axis=1)

### Normalize the nationality and add the iso-3166-1 alpha 2 codes

In [174]:
def map_country(nationality: str, language: str, mapping: dict) -> str:
    if nationality:
        if language == 'de':
            nationality = nationality.split()[0].strip()
        else:
            nationality = nationality.split()[-1].strip()
        return mapping[language].get(nationality)
    return ''

In [107]:
nat_cols = [col for col in df_people_concat.columns if re.match(r'^nationality\_\d{1}$', col)]
for nat_col in nat_cols:
    df_people_concat[f"{nat_col}_norm"] = df_people_concat[nat_col].fillna('').apply(normalize_words)
    df_people_concat[f'{nat_col}_iso_3166_1_alpha_2'] = df_people_concat[f"{nat_col}_norm"].apply(lambda x: map_country(x, 'fr', nationality_mapping))

In [108]:
df_people_concat

,ehraid,message_raw,shab_date,shab_id,codes,keyword,first_name,last_name,job_title,authorization,...,hometown_2,hometown_3,place_of_residence_1,place_of_residence_2,nationality_1,nationality_2,nationality_1_norm,nationality_2_norm,nationality_1_iso_3166_1_alpha_2,nationality_2_iso_3166_1_alpha_2
0,252,"BG SA, à Bagnes, CHE-103.527.669, société anon...",2017-05-17,3527803,[aenderungorgane],personnes et signatures radiées,Jacques,Bruchez,administrateur et secrétaire,avec signature individuelle,...,None,None,Martigny,None,CH,None,ch,,None,
1,252,"BG SA, à Bagnes, CHE-103.527.669, société anon...",2017-05-24,3541469,[aenderungorgane],inscription ou modification de personnes,Eric,Fumeaux,administrateur unique,avec signature individuelle,...,None,None,Le Châble VS ( Bagnes ),None,CH,None,ch,,None,
2,252,"BG SA, à Bagnes, CHE-103.527.669, société anon...",2020-12-28,1005060112,[aenderungorgane],personnes et signatures radiées,Eric,Fumeaux,administrateur unique,avec signature individuelle,...,None,None,Le Châble VS ( Bagnes ),None,CH,None,ch,,None,
3,252,"BG SA, à Bagnes, CHE-103.527.669, société anon...",2020-12-28,1005060112,[aenderungorgane],inscription ou modification de personnes,Sandra,De Vito,directrice,avec signature collective à deux,...,None,None,Lausanne,None,CH,None,ch,,None,
4,252,"BG SA, à Bagnes, CHE-103.527.669, société anon...",2020-12-28,1005060112,[aenderungorgane],inscription ou modification de personnes,Véronique,Billod,directrice adjointe,avec signature collective à deux,...,None,None,Sâles,None,CH,None,ch,,None,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120655,1681999,"SalvaTerre Ryan Van Boven, à Cugy (VD), Chemin...",2025-03-04,1006273021,"[status, status.neu]",personnes inscrites special,Boven Ryan,Van,titulaire,avec signature individuelle,...,None,None,Montanaire,None,Canada,None,canada,,CA,
120656,1682003,"Tserman Vladyslav, Ingénieur logiciel (Tserman...",2025-03-04,1006273026,"[status, status.neu]",personnes inscrites special,Vladyslav,Tserman,titulaire,avec signature individuelle,...,None,None,Vevey,None,Ukraine,None,ukraine,,UA,
120657,1682005,"Yaffa Nettoyages, Flamur Pllana, à Ecublens (V...",2025-03-04,1006273029,"[status, status.neu]",personnes inscrites special,Flamur,Pllana,titulaire,avec signature individuelle,...,None,None,Ecublens ( VD ),None,CH,None,ch,,None,
120658,1682009,"Clément Peyré, Cicatriclem, à Valbirse, CHE-14...",2025-03-04,1006272333,"[status, status.neu]",personnes inscrites,Clément,Peyré,titulaire,avec signature individuelle,...,None,None,Malleray ( Valbirse ),None,ressortissant français,None,ressortissant francais,,FR,


### Find gendered job titles and/or determine gender

In [16]:
gendered_endings = {
    'de': {
        'female': ['in'],
        'male': [],  # no specific ending for male words in German
    },
    'fr': {
        'female': ['euse', 'ienne', 'onne', 'ane', 'trice', 'esse'],
        'male': ['eur', 'ien', 'on', 'an'],
    },
    'it': {
        'female': ['a', 'trice', 'essa'],
        'male': ['o', 'ore']
    }   
}


def extract_nouns(text: str, language: str) -> list:
    """
    Extracts nouns from a given text using spaCy for German, French, and Italian.
    """
    nlp = nlp_models[language]
    return [token.text for token in nlp(text) if token.pos_ == 'NOUN']


def create_gendered_job_names(language: str, df: pd.DataFrame, col: str = 'job_title_norm') -> tuple[list]:
    nouns = set([title.lower() for title in df[col].unique() for title in extract_nouns(title, language)])
       
    female_words = []
    male_words = []
    undetermined = []

    for word in nouns:
        if any(word.endswith(ending) for ending in gendered_endings[language]['female']):
            female_words.append(word)
        elif any(word.endswith(ending) for ending in gendered_endings[language]['male']):
            male_words.append(word)
        else:
            undetermined.append(word)

    if language == 'de':
        for word in female_words:
            male_version = word.removesuffix('in')
            if male_version in undetermined:
                male_words.append(male_version)
        
    undetermined = [w for w in undetermined if w not in male_words]

    return female_words, male_words, undetermined

In [ ]:
# Normalize job title
df_people_concat['job_title_norm'] = df_people_concat['job_title'].apply(normalize_words)
df_people_concat['job_title_norm'] = df_people_concat['job_title_norm'].str.replace(r'[^a-zA-Z]', ' ', regex=True).apply(lambda x: ' '.join(x.split()))

In [ ]:
CREATE_GENDERED_WORDS = False

if CREATE_GENDERED_WORDS:
    female_words, male_words, undetermined = create_gendered_job_names('fr', df_people_concat)

In [131]:
def determine_gender(mapping: dict, nationalities: list[str], job_title: str, name: str, country_code):
    """
    """
    # Try to infer gender via nationality
    for nationality in nationalities:
        if re.search(r'\bstaatsangehoeriger\b|\bcittadino\b|\bressortissant\b|\bcitoyen\b', nationality):
            return 'm'
        elif re.search(r'\bstaatsangehoerige\b|\bcittadina\b|\bressortissante\b|\bcitoyenne\b', nationality):
            return 'f'

    # Try to infer gender via job title
    genders = [mapping[w] for w in mapping.keys() if re.match(rf'\b{w}\b', job_title)]
    if genders:
        # Check if list only contains one gender
        if genders.count(genders[0]) == len(genders):
            return genders[0]

    # Try to infer gender via first name and country_code
    pass

    return 'u'

In [132]:
nat_norm_cols = [col for col in df_people_concat.columns if re.match(r'\bnationality_\d{1}_norm\b', col)]
df_people_concat['gender'] = df_people_concat.apply(lambda x: determine_gender(gender_mapping['fr'], [x[col] for col in nat_norm_cols], x['job_title_norm'], x['first_name'], x['nationality_1_iso_3166_1_alpha_2']), axis=1)

# PROCESS CAPITAL CHANGES

In [ ]:
capital_changes = parsed_shab_messages[parsed_shab_messages.main_group == 'kapital- und rechtsformänderungen'].copy()
mergers_and_acquisitions = parsed_shab_messages[parsed_shab_messages.main_group == 'fusionen und ab- und aufspaltungen'].copy()

In [ ]:
# Correct wrong values like '446.001.000.00'
def correct_number(number_str: str) -> str:
    parts = number_str.rsplit('.', 1)
    return parts[0].replace('.', '') + '.' + parts[1] if len(parts) > 1 else parts[0].replace('.', '')

In [ ]:
shab_ids = [[], []]
keywords = [[], []]
main_groups = [[], []]

capital_new = []
capital_until_now = []
num_shares_new = []
val_shares_new = []
typ_shares_new = []

for i, row in capital_changes.iterrows():
    shab_id = row['shab_id']
    keyword = row['keyword']
    main_group = row['main_group']

    parsed_variables = row['parsed_variables']
    cap_new = parsed_variables.get('capital_new', [])
    cap_unt = parsed_variables.get('capital_until_now', [])
    srs_new = parsed_variables.get('shares_new', [])

    if cap_new or cap_unt:
        capital_new.append(cap_new[0] if len(cap_new) > 0 else None)
        capital_until_now.append(cap_unt[0] if len(cap_unt) > 0 else None)
        shab_ids[0].append(shab_id)
        keywords[0].append(keyword)
        main_groups[0].append(main_group)
    if srs_new:
        for s in srs_new:
            num_shares_new.append(s.get('number'))
            val_shares_new.append(s.get('value'))
            typ_shares_new.append(s.get('type'))
            shab_ids[1].append(shab_id)
            keywords[1].append(keyword)
            main_groups[1].append(main_group)

In [ ]:
cap_new = pd.DataFrame({
    'shab_id': shab_ids[0],
    'keyword': keywords[0],
    'main_group': main_groups[0],
    'capital_new': capital_new,
    'capital_until_now': capital_until_now})

cap_new['capital_new'] = cap_new['capital_new'].fillna('').str.replace("'", "", regex=False)
cap_new['capital_until_now'] = cap_new['capital_until_now'].fillna('').str.replace("'", "", regex=False)

# Extract Währung
cap_new['currency_new'] = cap_new['capital_new'].str.extract(r'^([^\d\s]+)')
cap_new['currency_new'] = cap_new['currency_new'].fillna('')

cap_new['currency_until_now'] = cap_new['capital_until_now'].str.extract(r'^([^\d\s]+)')
cap_new['currency_until_now'] = cap_new['currency_until_now'].fillna('')

# Extract Kapital
cap_new['capital_new'] = cap_new['capital_new'].str.extract(r'([\d.,]+)').astype(str)
cap_new['capital_until_now'] = cap_new['capital_until_now'].str.extract(r'([\d.,]+)').astype(str)

# Apply correction to remove unneccessary punctuations
cap_new['capital_new'] = cap_new['capital_new'].apply(correct_number)
cap_new['capital_until_now'] = cap_new['capital_until_now'].apply(correct_number)

cap_new['capital_new'] = [v.replace(',', '.') if not '.' in v else v.replace(',', '') for v in cap_new['capital_new']]
cap_new.loc[cap_new['capital_new'] == '.', 'capital_new'] = np.nan

cap_new['capital_until_now'] = [v.replace(',', '.') if not '.' in v else v.replace(',', '') for v in cap_new['capital_until_now']]
cap_new.loc[cap_new['capital_until_now'] == '.', 'capital_until_now'] = np.nan

# Ensure correct types
cap_new['capital_new'] = cap_new['capital_new'].astype(float)
cap_new['capital_until_now'] = cap_new['capital_until_now'].astype(float)

In [ ]:
stocks_new = pd.DataFrame({
    'shab_id': shab_ids[1],
    'keyword': keywords[1],
    'hauptkategorie': main_groups[1],
    'num_shares_new': num_shares_new,
    'val_shares_new': val_shares_new,
    'typ_shares_new': typ_shares_new,})

stocks_new['val_shares_new'] = stocks_new['val_shares_new'].fillna('').str.replace("'", "", regex=False)

# Extract Währung
stocks_new['currency_shares_new'] = stocks_new['val_shares_new'].str.extract(r'^([^\d\s]+)')
stocks_new['currency_shares_new'] = stocks_new['currency_shares_new'].fillna('')

# Extract number of Stocks, etc.
stocks_new['val_shares_new'] = stocks_new['val_shares_new'].str.extract(r'([\d.,]+)').astype(str)

# Apply correction to remove unneccessary punctuations
stocks_new['val_shares_new'] = stocks_new['val_shares_new'].apply(correct_number)
stocks_new['val_shares_new'] = [v.replace(',', '.') if not '.' in v else v.replace(',', '') for v in stocks_new['val_shares_new']]
stocks_new.loc[stocks_new['val_shares_new'] == '.', 'val_shares_new'] = np.nan

# Ensure correct types
stocks_new['val_shares_new'] = stocks_new['val_shares_new'].astype(float)
stocks_new['num_shares_new'] = stocks_new['num_shares_new'].astype(int)

In [ ]:
# Calculate total value of the capital by multiplying the number of shares with their individual value
stocks_new['value_total'] = stocks_new['num_shares_new'] * stocks_new['val_shares_new']

# Calculate new capital for shab ids where kapital_neu variable is not given, but scheine_neu is
missing_ids = set(stocks_new.shab_id).difference(set(cap_new.shab_id))
stocks_new_missing = stocks_new[stocks_new.shab_id.isin(missing_ids)]

cap_new_missing = stocks_new_missing.groupby(['shab_id', 'keyword', 'main_group']).agg(
    capital_new=pd.NamedAgg(column='value_total', aggfunc='sum'),
    currency_new=pd.NamedAgg(column='currency_shares_new', aggfunc=lambda x: list(set([currency for currency in x if currency != ''])))).reset_index()

In [ ]:
# Check if there are mixed currencies
assert len([cur_set for cur_set in cap_new_missing['currency_new'] if len(cur_set) > 1]) == 0

cap_new_missing['currency_new'] = [v[0] if len(v) > 0 else '' for v in cap_new_missing['currency_new']]
cap_new_missing['currency_until_now'] = ''
cap_new_missing['capital_new'] = np.nan

In [ ]:
# Combine the two dataframes to get all capital changes
cap_new_concat = pd.concat([cap_new, cap_new_missing])

In [ ]:
cap_new_concat.head()

In [ ]:
cap_new_concat.to_csv(EXTERNAL_DATA_DIR / 'capital_changes.csv', index=False)

# PROCESS MERGERS AND ACQUISIITONS

In [ ]:
mergers_and_acquisitions

In [ ]:
shab_ids = []
keywords = []
main_groups = []
text_passages = []

contract_dates = []
bilanzdaten = []

firm_taken_over = []
location_taken_over = []
id_taken_over = []
assets_taken_over = []
liabilities_taken_over = []

for i, row in mergers_and_acquisitions.iterrows():
    shab_id = row['shab_id']
    keyword = row['keyword']
    main_group = row['main_group']
    text_passage = row['text_slice']

    parsed_variables = row['parsed_variables']

    if parsed_variables:
        contract_date = parsed_variables.get('contract_date', [])
        bilanzdatum = parsed_variables.get('bilanzdatum', [])
        firms_taken_over = parsed_variables.get('firms_taken_over', [])
        for firm in firms_taken_over:
            firm_name = firm.get('firm_name', '')
            location = firm.get('location', '')
            id = firm.get('id', '')
            capital_taken_over = firm.get('capital_taken_over', {})
            assets = capital_taken_over.get('aktiven', '') if capital_taken_over else ''
            liabilities = capital_taken_over.get('passiven', '') if capital_taken_over else ''

            contract_dates.append(contract_date[0] if len(contract_date) > 0 else '')
            bilanzdaten.append(bilanzdatum[0] if len(bilanzdatum) > 0 else '')
            firm_taken_over.append(firm_name)
            location_taken_over.append(location)
            id_taken_over.append(id)
            assets_taken_over.append(assets)
            liabilities_taken_over.append(liabilities)
            shab_ids.append(shab_id)
            keywords.append(keyword)
            main_groups.append(main_group)
            text_passages.append(text_passage)
    else:
        firm_taken_over.append('')
        location_taken_over.append('')
        id_taken_over.append('')
        assets_taken_over.append('')
        liabilities_taken_over.append('')
        shab_ids.append(shab_id)
        keywords.append(keyword)
        main_groups.append(main_group)
        text_passages.append(text_passage)

In [ ]:
assert len(shab_ids) == len(keywords) == len(main_groups) == len(firm_taken_over) == len(location_taken_over) == len(id_taken_over) == len(assets_taken_over) == len(liabilities_taken_over)

In [ ]:
processed_mergers = pd.DataFrame({
    'shab_id': shab_ids,
    'keyword': keywords,
    'hauptkategorie': main_groups,
    'texte': text_passages,
    'firm_taken_over': firm_taken_over,
    'location_taken_over': location_taken_over,
    'id_taken_over': id_taken_over,
    'assets_taken_over': assets_taken_over,
    'liabilities_taken_over': liabilities_taken_over
})

processed_mergers['assets_taken_over'] = processed_mergers['assets_taken_over'].fillna('').str.replace("'", "", regex=False)
processed_mergers['liabilities_taken_over'] = processed_mergers['liabilities_taken_over'].fillna('').str.replace("'", "", regex=False)

# Extract Währung of Aktiven/Passiven
processed_mergers['currency_assets_taken_over'] = processed_mergers['assets_taken_over'].str.extract(r'^([^\d\s]+)')
processed_mergers['currency_assets_taken_over'] = processed_mergers['currency_assets_taken_over'].fillna('')

processed_mergers['currency_liabilities_taken_over'] = processed_mergers['liabilities_taken_over'].str.extract(r'^([^\d\s]+)')
processed_mergers['currency_liabilities_taken_over'] = processed_mergers['currency_liabilities_taken_over'].fillna('')

# Extract value of Aktiven/Passiven
processed_mergers['assets_taken_over'] = processed_mergers['assets_taken_over'].str.extract(r'([\d.,]+)').astype(str)

processed_mergers['liabilities_taken_over'] = processed_mergers['liabilities_taken_over'].str.extract(r'([\d.,]+)').astype(str)

# Apply correction to remove unneccessary punctuations
processed_mergers['assets_taken_over'] = processed_mergers['assets_taken_over'].apply(correct_number)
processed_mergers['assets_taken_over'] = [v.replace(',', '.') if not '.' in v else v.replace(',', '') for v in processed_mergers['assets_taken_over']]

processed_mergers['liabilities_taken_over'] = processed_mergers['liabilities_taken_over'].apply(correct_number)
processed_mergers['liabilities_taken_over'] = [v.replace(',', '.') if not '.' in v else v.replace(',', '') for v in processed_mergers['liabilities_taken_over']]

# Ensure correct types
processed_mergers['assets_taken_over'] = processed_mergers['assets_taken_over'].astype(float)
processed_mergers['liabilities_taken_over'] = processed_mergers['liabilities_taken_over'].astype(float)

In [ ]:
processed_mergers.head()

In [ ]:
processed_mergers.to_csv(EXTERNAL_DATA_DIR / 'merger_sizes.csv', index=False)